#### **Fase 2: Procesamiento, Normalización y Carga (ETL)**

**Objetivo:** Transformar el dataset unificado (`dataset_unificado.csv`) en una estructura relacional que cumpla con la **3NF** para su posterior volcado en una base de datos SQL utilizando **SQLAlchemy**.

---

El proceso seguirá estos pasos:

1. **Lectura y Limpieza:**
    * Carga de los datos integrados de Spotify y Last.fm.
    * Tratamiento y estandarización final de tipos de datos.

2. **Identificación de Entidades:**
    * Separación del dataframe maestro en entidades independientes y únicas: **Álbumes, Artistas, Tracks, Géneros y Tags**.

3. **Gestión de Relaciones N:M (Muchos a Muchos):**
    * Construcción de las tablas intermedias para normalizar esas relaciones.

4. **Mapeo de IDs (ID Management):**
    * Carga secuencial en SQL respetando la jerarquía de dependencias.
    * Recuperación de IDs autoincrementales generados por la BD para vincular las tablas mediante *Foreign Keys*.

---

In [ ]:
#Importamos pandas para la lectura y manipulacion de la información:
import pandas as pd
pd.set_option('display.max_columns', None)

# Cargamos el dataset unificado y el maestro de artistas generados en la fase 1 para iniciar la fase de transformación:
df_unificado = pd.read_csv('dataset_unificado.csv')
df_maestro_artistas = pd.read_csv('maestro_artistas.csv')

##### **1. Creación de tablas principales**

In [ ]:
# --- ENTIDAD: ALBUM ---

# Extraemos álbumes únicos basándonos en el ID de Spotify para evitar duplicidad de títulos.
df_album = df_unificado[['album_id', 'album_name', 'album_type', 'total_tracks', 'album_release_date', 'label']].drop_duplicates(subset=['album_id']).copy()
df_album.info()

In [ ]:
# Limpiamos registros nulos en 'album_name' para cumplir con la restricción NOT NULL de la base de datos.
df_album2 = df_album.dropna(subset=['album_name'])
df_album2.drop(columns=['album_id'], inplace=True)
df_album2.info()

In [ ]:
# --- ENTIDAD: ARTIST ---

# Consolidamos la información del artista principal y colaboradores.
# Renombramos columnas para asegurar la consistencia durante el cruce de datos (merge).
df_nombres_ids = df_unificado[['artist', 'artistid']].drop_duplicates(subset=['artistid'])
df_nombres_ids.rename(columns={'artist': 'artist_name', 'artistid': 'artist_id'}, inplace=True)

# Unimos la información de ambos archivos
df_artist = pd.merge(
    df_nombres_ids, 
    df_maestro_artistas, 
    on='artist_id', 
    how='left'
)

# Selección de los atributos de la tabla artist
df_artist = df_artist[['artist_name', 'artist_popularity', 'artist_followers']]
df_artist.info()

In [ ]:
# --- ENTIDAD: GENRE ---

# Aplicamos normalización (1NF): 'atomizamos' las listas de géneros para que cada registro sea único.
# Esto convierte strings separados por comas en filas independientes y ordenadas.
generos_unicos = df_maestro_artistas['artist_genres'].str.split(',').explode().str.strip().dropna().unique()

# Creamos el DataFrame ordenado alfabéticamente
df_genre = pd.DataFrame({'genre_name': sorted(list(generos_unicos))})
df_genre.info()

In [ ]:
# --- ENTIDAD: TAG ---

# Aplicamos normalización (1NF): 'atomizamos' las listas de tags para que cada registro sea único.
# Como aparecen tags extraños, limpiamos y evitamos tags imprecisos que no aporten valor.
tags_unicos = df_unificado['tags'].str.split(',').explode().str.strip().dropna().unique()
tags_filtrados = [t for t in tags_unicos if len(str(t)) > 2 and not str(t).isdigit()]

# Creamos el DataFrame ordenado alfabéticamente
df_tags = pd.DataFrame({'tags': sorted(list(tags_filtrados))})
df_tags.info()

In [ ]:
# --- ENTIDAD: TRACK ---

# Seleccionamos los atributos. Mantenemos 'album_name' temporalmente para el mapeo de llaves foráneas.
df_tracks = df_unificado[[
    'track', 'track_release_date', 'track_popularity', 
    'track_year', 'collaboration', 'genre_extracted', 
    'listeners', 'playcount', 'artist', 'album_id', 'album_name'
]].copy()

df_tracks.rename(columns={'track': 'track_name'}, inplace=True)

df_tracks.info()

##### **2. Volcado a SQL y gestión de llaves foráneas (FK)**

In [ ]:
import mysql.connector
from mysql.connector import errorcode
from sqlalchemy import create_engine,FLOAT, VARCHAR, INTEGER, DATE, SmallInteger
from sqlalchemy.sql.sqltypes import String

In [ ]:
# Establecemos la conexión con la BD
mysql_user = input("Introduce tu usuario de MySQL: ")
mysql_password = input("Introduce tu contraseña de MySQL: ")
mysql_host = input("Introduce tu host de MySQL: ")

try:
    engine = create_engine(f"mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}/rhythmiq")
    print("Conexión establecida con éxito")
except Exception as e:
    print(f"Ocurrió un error: {e}")

In [ ]:
# Volcado de ALBUM: Es necesario cargar esta tabla primero para generar los IDs que usará la tabla Track
try:
    df_album2.to_sql('album', engine, if_exists='append', index=False) 
    print("Datos insertados correctamente.")
    print(f"Se han insertado {len(df_album2)} registros.")

except Exception as e:
    print(f"Ocurrió un error: {e}")

In [ ]:
# Mapeo de llaves foráneas:
# Leemos los IDs generados automáticamente por SQL para vincular correctamente cada canción con su álbum:

df_album_db = pd.read_sql("SELECT album_id, album_name FROM album", engine)
df_album_db_clean = df_album_db.drop_duplicates(subset=['album_name'])

# Cruzamos la tabla de canciones con la de álbumes para obtener el ID numérico (Foreign Key)
df_track2 = pd.merge(df_tracks, df_album_db_clean, on='album_name', how='left')

# Limpieza final de la tabla TRACK: Eliminamos columnas redundantes y renombramos la FK
df_track2.rename(columns={'album_id_y': 'album_id'}, inplace=True)
df_track_final = df_track2.drop(columns=['artist', 'album_id_x', 'album_name'])
df_track_final


In [ ]:
# Volcado de ARTIST
try:
    df_artist.to_sql('artist', engine, if_exists='append', index=False) 
    print("Datos insertados correctamente.")
    print(f"Se han insertado {len(df_artist)} registros.")

except Exception as e:
    print(f"Ocurrió un error: {e}")

In [ ]:
# Volcado de GENRE
try:
    df_genre.to_sql('genre', engine, if_exists='append', index=False) 
    print("Datos insertados correctamente.")
    print(f"Se han insertado {len(df_genre)} registros.")

except Exception as e:
    print(f"Ocurrió un error: {e}")

In [ ]:
# Volcado de TAG
try:
    df_tags.to_sql('tag', engine, if_exists='append', index=False) 
    print("Datos insertados correctamente.")
    print(f"Se han insertado {len(df_tags)} registros.")

except Exception as e:
    print(f"Ocurrió un error: {e}")

In [ ]:
# Volcado de TRACK
try:
    df_track_final.to_sql('track', engine, if_exists='append', index=False) 
    print("Datos insertados correctamente.")
    print(f"Se han insertado {len(df_track_final)} registros.")

except Exception as e:
    print(f"Ocurrió un error: {e}")

##### **3. Generación de Tablas Intermedias y Relaciones (N:M)**

In [ ]:
# Mapeo de llaves foráneas de todas las tablas creadas:
# Leemos los IDs generados automáticamente por SQL para vincular correctamente cada tabla:

df_artist_db = pd.read_sql("SELECT artist_id, artist_name FROM artist", engine)
df_genre_db = pd.read_sql("SELECT genre_id, genre_name FROM genre", engine)
df_tag_db = pd.read_sql("SELECT tag_id, tags FROM tag", engine)
df_track_db = pd.read_sql("SELECT track_id, track_name FROM track", engine)

In [ ]:
# --- TABLA INTERMEDIA: TRACK_ARTIST ---
# Relaciona cada canción con todos sus artistas (principales y colaboradores).

df_track_artist = (df_unificado[['track', 'artist', 'all_artists_names']]
    .assign(all_artists_names=df_unificado['all_artists_names'].str.split(','))
    .explode('all_artists_names')
    .dropna()
)

df_track_artist['all_artists_names'] = df_track_artist['all_artists_names'].str.strip()

df_track_artist['is_main_artist'] = (df_track_artist['all_artists_names'] == df_track_artist['artist']).astype(int)

# Cruzamos con los IDs de SQL
df_inter_track_artist = pd.merge(
    df_track_artist, 
    df_track_db, 
    left_on='track', 
    right_on='track_name'
)

df_inter_track_artist = pd.merge(
    df_inter_track_artist, 
    df_artist_db, 
    left_on='all_artists_names', 
    right_on='artist_name'
)

# Limpiamos y volcamos
df_final_track_artist = (df_inter_track_artist[['track_id', 'artist_id', 'is_main_artist']].drop_duplicates(subset=['track_id', 'artist_id']))

try:
    df_final_track_artist.to_sql('track_artist', engine, if_exists='append', index=False)
    print("Datos insertados correctamente.")
    print(f"Se han insertado {len(df_final_track_artist)} registros.")
except Exception as e:
    print(f"Ocurrió un error: {e}")

In [ ]:
# --- TABLA INTERMEDIA: ARTIST_GENRE ---
# Relaciona a los artistas con sus estilos musicales.

mapa_nombres = df_unificado[['artistid', 'artist']].drop_duplicates()
mapa_nombres.columns = ['artist_id', 'artist_name']

df_art_gen = (df_maestro_artistas[['artist_id', 'artist_genres']]
              .assign(artist_genres=df_maestro_artistas['artist_genres'].str.split(','))
              .explode('artist_genres')
              .dropna())
df_art_gen['artist_genres'] = df_art_gen['artist_genres'].str.strip()

df_art_gen = pd.merge(df_art_gen, mapa_nombres, on='artist_id')
df_art_gen['artist_genres'] = df_art_gen['artist_genres'].str.strip()

# Cruzamos con los IDs de SQL

df_inter_art_gen = pd.merge(
    df_art_gen, 
    df_artist_db, 
    on='artist_name'
)

df_inter_art_gen = pd.merge(
    df_inter_art_gen, 
    df_genre_db, 
    left_on='artist_genres', 
    right_on='genre_name'
)

# Limpiamos y volcamos
df_final_art_gen = df_inter_art_gen[['artist_id_y', 'genre_id']].drop_duplicates()
df_final_art_gen.columns = ['artist_id', 'genre_id']

try:
    df_final_art_gen.to_sql('artist_genre', engine, if_exists='append', index=False)    
    print("Datos insertados correctamente.")
    print(f"Se han insertado {len(df_final_art_gen)} registros.")
except Exception as e:
    print(f"Ocurrió un error: {e}")

In [ ]:
# --- TABLA INTERMEDIA: TRACK_TAG ---
# Relaciona cada canción con todos sus artistas (principales y colaboradores).

df_track_tag = (df_unificado[['track', 'tags']]
                .assign(tags=df_unificado['tags'].str.split(','))
                .explode('tags')
                .dropna())
df_track_tag['tags'] = df_track_tag['tags'].str.strip()
df_track_tag['track'] = df_track_tag['track'].str.strip()

# Cruzamos con los IDs de SQL
df_inter_track_tag = pd.merge(
    df_track_tag, 
    df_track_db, 
    left_on='track', 
    right_on='track_name'
)

df_inter_track_tag = pd.merge(
    df_inter_track_tag, 
    df_tag_db, 
    left_on='tags', 
    right_on='tags'
)

# Limpiamos y volcamos
df_final_track_tag = df_inter_track_tag[['track_id', 'tag_id']].drop_duplicates()

try:
    df_final_track_tag.to_sql('track_tag', engine, if_exists='append', index=False)
    print("Datos insertados correctamente.")
    print(f"Se han insertado {len(df_final_track_tag)} registros.")
except Exception as e:
    print(f"Ocurrió un error: {e}")